In [7]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.tools import tool
import os
import matplotlib.pyplot as plt
import seaborn as sns
import io
import base64
import datetime
import json
from typing import Optional

In [2]:
# --- Configuración de OpenAI ---
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') 

# Inicializa el LLM. GPT-4o es excelente para esto.
llm = ChatOpenAI(model="gpt-4o", temperature=0.0) # Temperatura baja para respuestas más deterministas

In [3]:
# Variable global para el DataFrame. Se inicializará en main.
df_global = None 

In [9]:
# --- Función para generar la gráfica ---
# Ahora la marcamos con @tool para que LangChain la reconozca como una herramienta.
@tool # Decorador para registrar la función como herramienta
def create_and_save_plot(
    plot_type: str = 'line',
    x_col: str = 'Fecha',
    y_col: str = 'Ventas',
    hue_col: Optional[str] = None, # ¡CAMBIO AQUÍ! Ahora acepta None como tipo válido
    title: str = 'Ventas a lo Largo del Tiempo',
    filters: Optional[dict] = None # También Optional para filters por si viene None
) -> str:
    """
    Genera y guarda una gráfica de los datos del DataFrame global.
    El agente puede usar esta herramienta cuando se le pide una visualización.

    Parámetros:
    plot_type (str): Tipo de gráfica a generar ('line' o 'bar').
    x_col (str): Nombre de la columna para el eje X.
    y_col (str): Nombre de la columna para el eje Y.
    hue_col (Optional[str]): Nombre de la columna para diferenciar series (ej. 'Country' o 'Coffee type'). Puede ser None.
    title (str): Título de la gráfica.
    filters (Optional[dict]): Un diccionario de filtros a aplicar al DataFrame antes de graficar.
                    Debe ser un diccionario con pares clave-valor.
                    Los valores pueden ser un string simple (ej. {'Country': 'Angola'})
                    o una lista de strings para múltiples valores (ej. {'Country': ['Colombia', 'Angola']}).
                    Si no se necesitan filtros, pasa un diccionario vacío {}.

    Retorna:
    str: Un mensaje indicando el éxito y la ruta del archivo de la gráfica, o un mensaje de error.
    """
    global df_global

    if df_global is None:
        return "Error: DataFrame no cargado. No se puede generar la gráfica."

    df_data = df_global.copy()

    # --- Manejo del parámetro filters (sin cambios en la lógica interna) ---
    if filters:
        if not isinstance(filters, dict):
            try:
                filters = json.loads(filters)
                if not isinstance(filters, dict):
                    return f"Error: El parámetro 'filters' debe ser un diccionario válido. Se recibió: {filters}"
            except (json.JSONDecodeError, TypeError):
                return f"Error: El parámetro 'filters' debe ser un diccionario. Se recibió un tipo no compatible: {type(filters).__name__}. Valor: {filters}"

        for col, val in filters.items():
            if col in df_data.columns:
                if isinstance(val, list):
                    df_data = df_data[df_data[col].isin(val)]
                elif pd.api.types.is_string_dtype(df_data[col]):
                    df_data = df_data[df_data[col].str.contains(val, case=False, na=False)]
                else:
                    df_data = df_data[df_data[col] == val]
            else:
                return f"Error: Columna de filtro '{col}' no encontrada en el DataFrame."

    # Asegúrate de que 'Fecha' sea datetime y 'Ventas' sea numérico
    if 'Fecha' in df_data.columns:
        df_data['Fecha'] = pd.to_datetime(df_data['Fecha'], errors='coerce')
        df_data = df_data.dropna(subset=['Fecha'])
    if 'Ventas' in df_data.columns:
        df_data['Ventas'] = pd.to_numeric(df_data['Ventas'], errors='coerce').fillna(0)

    # Validar que las columnas necesarias existan
    required_cols = [x_col, y_col]
    # Solo añadir hue_col a required_cols si no es None
    if hue_col: 
        required_cols.append(hue_col)

    if not all(col in df_data.columns for col in required_cols):
        return f"Error: Una o más columnas requeridas para graficar no encontradas: X='{x_col}', Y='{y_col}'" + (f", Agrupar='{hue_col}'" if hue_col else "") + ". Verifica los nombres de las columnas."

    # Eliminar filas con valores NaN en las columnas clave para evitar errores de graficado
    df_data = df_data.dropna(subset=required_cols)

    if df_data.empty:
        return "No hay datos para graficar después de aplicar los filtros o el preprocesamiento."

    plt.figure(figsize=(10, 6))

    if plot_type == 'line':
        sns.lineplot(data=df_data, x=x_col, y=y_col, hue=hue_col)
    elif plot_type == 'bar':
        if hue_col:
            grouped_data = df_data.groupby([x_col, hue_col])[y_col].sum().reset_index()
            sns.barplot(data=grouped_data, x=x_col, y=y_col, hue=hue_col, estimator='sum', errorbar=None)
        else:
            grouped_data = df_data.groupby(x_col)[y_col].sum().reset_index()
            sns.barplot(data=grouped_data, x=x_col, y=y_col, estimator='sum', errorbar=None)
    else:
        plt.close()
        return f"Tipo de gráfica '{plot_type}' no soportado. Usa 'line' o 'bar'."

    plt.title(title)
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '_')).replace(' ', '_')[:50]
    file_name = f"grafica_{safe_title}_{timestamp}.png"
    file_path = os.path.join(os.getcwd(), file_name)

    try:
        plt.savefig(file_path, format='png', bbox_inches='tight')
        plt.close()
        return f"Gráfica guardada exitosamente en: {file_name}"
    except Exception as e:
        plt.close()
        return f"Error al guardar la gráfica: {e}"

In [10]:
# --- Función Principal ---
def main():
    """
    Función principal para cargar y preprocesar un archivo CSV, configurar un agente de Pandas DataFrame,
    y permitir consultas interactivas de datos, incluyendo solicitudes de visualización.

    Flujo general:
    1. Cargar y limpiar los datos desde un archivo CSV.
    2. Configurar un agente de Pandas DataFrame con herramientas adicionales.
    3. Permitir consultas del usuario sobre los datos y responderlas dinámicamente.
    4. Manejar solicitudes de gráficos de manera automática.
    
    Retorna:
    None: La función es interactiva y no devuelve valores.
    """
    global df_global # Indicar que vamos a modificar la variable global df_global

    csv_file_path = "coffee_final.csv"

    try:
        df_global = pd.read_csv(csv_file_path)
        # Preprocesar el DataFrame al cargarlo
        if 'Fecha' in df_global.columns:
            df_global['Fecha'] = pd.to_datetime(df_global['Fecha'], errors='coerce')
        if 'Ventas' in df_global.columns:
            df_global['Ventas'] = pd.to_numeric(df_global['Ventas'], errors='coerce')
            df_global['Ventas'] = df_global['Ventas'].fillna(0) 
        df_global = df_global.dropna(subset=['Fecha', 'Ventas']) # Eliminar filas con NaT en Fecha o NaN en Ventas
        
        print(f"DataFrame cargado con {len(df_global)} filas.")
        print("Columnas disponibles:", df_global.columns.tolist())
    except FileNotFoundError:
        print(f"Error: El archivo '{csv_file_path}' no se encontró. Asegúrate de que está en la misma carpeta o la ruta es correcta.")
        return
    except Exception as e:
        print(f"Error al cargar o preprocesar el CSV: {e}")
        return

    # Definir las herramientas que el agente puede usar
    # El agente de Pandas DataFrame ya tiene la herramienta PythonAstREPLTool implícita para manipular el DF.
    # Ahora le añadimos explícitamente nuestra herramienta de visualización.
    tools = [
        create_and_save_plot # Nuestra herramienta personalizada
    ]

    # Crear el agente de Pandas DataFrame con las herramientas adicionales
    pandas_agent = create_pandas_dataframe_agent(
        llm,
        df_global, # Le pasamos el DataFrame completo
        verbose=True, # Para ver el proceso de pensamiento del agente
        agent_type="tool-calling", # Usa las capacidades de tool-calling de OpenAI
        allow_dangerous_code=True, # ¡Usar con precaución en producción!
        extra_tools=tools # Añadir nuestras herramientas personalizadas
    )

    print("\n¡Agente de Pandas DataFrame con herramienta de visualización listo!")
    print("Puedes hacer preguntas sobre los datos, solicitar cálculos o pedir visualizaciones.")
    print("Escribe 'salir' para terminar.")
    
    while True:
        query = input("\nTu pregunta: ")
        if query.lower() == 'salir':
            print("¡Hasta luego!")
            break
        
        try:
            # Aquí la magia ocurre: el agente interpreta la consulta, razona qué herramienta usar,
            # y la ejecuta.
            # Le damos un prompt extra para que sepa cómo manejar las visualizaciones
            # y sepa que tiene una herramienta para ello.
            full_query_for_agent = f"""
            Considera la siguiente consulta del usuario. Si la consulta implica una visualización (ej. 'grafica', 'muestra un gráfico', 'compara ventas', 'visualiza'), usa la herramienta 'create_and_save_plot'. Asegúrate de inferir los parámetros correctos (plot_type, x_col, y_col, hue_col, title, filters) para la herramienta basándote en la consulta y en el DataFrame que te fue proporcionado.
            Si no es una solicitud de visualización, usa tu conocimiento sobre el DataFrame para responder a la pregunta.

            Consulta del usuario: {query}
            """

            response_agent = pandas_agent.invoke({"input": full_query_for_agent})
            
            print("\n--- Respuesta del Agente ---")
            print(response_agent["output"])
            print("-----------------------------\n")

        except Exception as e:
            print(f"Ocurrió un error al procesar la consulta: {e}")
            print("Asegúrate de que el LLM tenga permiso para ejecutar código y que los nombres de las columnas sean correctos.")


In [12]:
if __name__ == "__main__":
    main()

DataFrame cargado con 1430 filas.
Columnas disponibles: ['Fecha', 'Country', 'Ventas', 'Coffee type']

¡Agente de Pandas DataFrame con herramienta de visualización listo!
Puedes hacer preguntas sobre los datos, solicitar cálculos o pedir visualizaciones.
Escribe 'salir' para terminar.




> Entering new AgentExecutor chain...

Invoking: `create_and_save_plot` with `{'plot_type': 'line', 'x_col': 'Fecha', 'y_col': 'Ventas', 'hue_col': 'Country', 'title': 'Ventas de Colombia y Ecuador a lo Largo de los Años', 'filters': {'Country': ['Colombia', 'Ecuador']}}`


Gráfica guardada exitosamente en: grafica_Ventas_de_Colombia_y_Ecuador_a_lo_Largo_de_los_Año_20250613_152510.pngLa gráfica de las ventas de Colombia y Ecuador a lo largo de los años ha sido creada y guardada exitosamente. Puedes encontrarla en el archivo: `grafica_Ventas_de_Colombia_y_Ecuador_a_lo_Largo_de_los_Año_20250613_152510.png`.

> Finished chain.

--- Respuesta del Agente ---
La gráfica de las ventas de Colombia y Ecuador a lo largo de los años ha sido creada y guardada exitosamente. Puedes encontrarla en el archivo: `grafica_Ventas_de_Colombia_y_Ecuador_a_lo_Largo_de_los_Año_20250613_152510.png`.
-----------------------------

¡Hasta luego!
